In [1]:
print("Welcome to ChatBot")

Welcome to ChatBot


In [2]:
%pwd

'c:\\Users\\mannu\\OneDrive\\Desktop\\Mecdical_Chatbot\\research'

In [3]:
import os

In [4]:
try:
    os.chdir("../")  # Change directory safely
except FileNotFoundError:
    print("Directory not found, skipping chdir...")


In [5]:
%pwd

'c:\\Users\\mannu\\OneDrive\\Desktop\\Mecdical_Chatbot'

# Start Coding

In [6]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Extract Data from the PDF File

In [7]:
import os
import pickle

CACHE_FILE = "cached_pdfs.pkl"

def load_pdf_file(data):
    """Agar cache file exist karti hai toh direct load karo"""
    if os.path.exists(CACHE_FILE):
        print("Loading cached extracted data...")
        with open(CACHE_FILE, "rb") as f:
            return pickle.load(f)
    
    print("Extracting data from PDFs...")
    loader = DirectoryLoader(data,
                             glob="*.pdf", 
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    # Cache extracted data
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(documents, f)

    return documents



In [8]:
# def load_pdf_file(data):
#     loader=DirectoryLoader(data,
#                            glob="*.pdf",
#                            loader_cls=PyPDFLoader
#                            )
#     documents = loader.load()

#     return documents

In [9]:
# Call function
extracted_data = load_pdf_file(data="Data/")

Loading cached extracted data...


In [10]:
print(f"Number of extracted documents: {len(extracted_data)}")

Number of extracted documents: 4505


# Perform Chunking Operation

In [11]:
#Split the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
print("Length of text_chunks :",len(text_chunks))

Length of text_chunks : 39994


# Embedding Model

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
# Download the Embedding from Hugging Face
def download_hugging_face_embeddings():
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\mannu\AppData\Local\Temp\ipykernel_4840\660017290.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

# Checking Embedding model Work Properly or Not

In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length of query_result :",len(query_result))

Length of query_result : 384


In [17]:
from dotenv import load_dotenv
load_dotenv()

True

# -----------------------------------------------------

# Enviroment Variable

In [18]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

# Pinecone Setup

In [19]:
# Initialize Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
            name=index_name,
            dimension=384,  # Your embedding model dimension
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
                )
        )

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': 'f254fdaa303b513666c064f5b962dcb4', 'date': 'Sat, 29 Mar 2025 21:34:37 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [20]:
# Initialize Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"

# # Fetch the list of existing indexes
# existing_indexes = pc.list_indexes()
# print("Available indexes:", existing_indexes)  # Debugging output

# # Use existing index instead of creating a new one
# if index_name in existing_indexes:
#     print(f"Index '{index_name}' already exists. Using it.")
#     index = pc.Index(index_name)  # Just use the existing index
# else:
#     print(f"Creating new index: {index_name}")
#     pc.create_index(
#         name=index_name,
#         dimension=384,  # Embedding model dimension
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1")
#     )
#     print(f"Index '{index_name}' created successfully!")
#     index = pc.Index(index_name)  # Initialize new index

# print(f"Using index: {index_name}")


# Get list of existing indexes
existing_indexes = pc.list_indexes()

if index_name in existing_indexes:
    print(f"Index '{index_name}' already exists. Skipping creation.")
    index = pc.Index(index_name)  # Initialize existing index
else:
    print(f"Creating new index: {index_name}")
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Index '{index_name}' created successfully!")
    index = pc.Index(index_name)  # Initialize new index


Creating new index: medicalbot


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '74a5a71acec11122d27e294061c9d5ed', 'date': 'Sat, 29 Mar 2025 21:34:57 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [21]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)  

existing_indexes = pc.list_indexes()
print("Existing indexes:", existing_indexes)


Existing indexes: [{
    "name": "medicalbot",
    "dimension": 384,
    "metric": "cosine",
    "host": "medicalbot-de95dym.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]


In [22]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)  

index_name = "medicalbot"

# Use the existing index
index = pc.Index(index_name)
print(f"Using existing index: {index_name}")


Using existing index: medicalbot


# Enviroment SetUp

In [29]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

# Load or Create Pinecone Index

In [30]:
# Load or Create Pinecone Index
from langchain_pinecone import PineconeVectorStore
try:
    # Embed each chunk and Upsert into Embedding Model to Your Pinecone Index
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
        )
    print("Using existing embeddings in Pinecone.")
except Exception:
    # Embed each chunk and Upsert the Embedding Model
    print("No existing embeddings found. Creating new ones...")
    docsearch = PineconeVectorStore.from_documents(
        documents=text_chunks,
        index_name=index_name,
        embedding=embeddings
        )


Using existing embeddings in Pinecone.


In [ ]:
# ## Embed each chunk and Upsert the Embedding Model
# from langchain_pinecone import PineconeVectorStore
# docsearch = PineconeVectorStore.from_documents(
#     documents = text_chunks,
#     index_name = index_name,
#     embedding = embeddings,
# )

# Load Existing Index

In [30]:
# ## Embed each chunk and Upsert into Embedding Model to Your Pinecone Index
# from langchain_pinecone import PineconeVectorStore
# docsearch = PineconeVectorStore.from_existing_index(
#     index_name = index_name,
#     embedding=embeddings,
# )

In [31]:
docsearch

# Retriever

In [32]:
retriever = docsearch.as_retriever(search_type = "similarity",search_kwargs = {"k":5})

In [33]:
retriever_docs = retriever.invoke("What is Ace?")

In [34]:
retriever_docs

[Document(id='384e21be-4d56-4f0e-b1e0-c459e78dff30', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 246.0, 'page_label': '217', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\Medical_Book_.pdf', 'total_pages': 4505.0}, page_content='Purpose\nThe ACE test is used primarily to detect and\nmonitor the clinical course ofsarcoidosis (a disease\nthat affects many organs, especially the lungs), to differ-\nentiate between sarcoidosis and similar diseases, and to\ndelineate between active and inactive sarcoid disease.\nElevated ACE levels are also found in a number of\nother conditions, including Gaucher’s disease (a rare\nfamilial disorder of fat metabolism) andleprosy.\nPrecautions\nIt should be noted that people under 20 years of\nage normally have very high ACE levels. Decreased\nlevels may be seen in the condition of excess fat in the\nblood (hyperlipidemia). Drugs that may cause\ndecreased A

# Model

In [35]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.1")


C:\Users\mannu\AppData\Local\Temp\ipykernel_21656\529195970.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")



from langchain_openai import OpenAI

llm = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  # Explicitly pass API key
    temperature=0.4,
    max_tokens=500
)

# Complete RAG Chain Pipeline

In [36]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """ 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
{context}
"""


prompt = ChatPromptTemplate.from_messages(
    [
    ("system",system_prompt),
    ("human","{input}"),
  ]
)

In [38]:
question_answering_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answering_chain)

In [39]:
response = rag_chain.invoke({"input":"What is Ace?"})
print(response["answer"])


ACE stands for Angiotensin-Converting Enzyme, a protein that plays an important role in regulating blood pressure by converting angiotensin I to angiotensin II. This enzyme is involved in the renin/aldosterone mechanism, which controls blood pressure. Elevated ACE levels can be found in various conditions, including sarcoidosis and Gaucher's disease.
